In [14]:
# 라이브러리 불러오기

import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [15]:
# 1. 데이터 불러오기
# 절대경로
path = "C:\study\_data\ddarung\\"
# 상대경로
# path = "_data\ddarung"

# index_col = 0 은 0번째 있는 id 를 index로 구분해줌 
train_csv = pd.read_csv(path+ "train.csv",index_col=0)
test_csv = pd.read_csv(path+"test.csv",index_col=0)
submission = pd.read_csv(path + "submission.csv",index_col=0)

In [16]:
# 칼럼명 출력
print(train_csv.columns)

Index(['hour', 'hour_bef_temperature', 'hour_bef_precipitation',
       'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility',
       'hour_bef_ozone', 'hour_bef_pm10', 'hour_bef_pm2.5', 'count'],
      dtype='object')


In [17]:

# 결측치 이상치를 확인해보기위한 카운트 조회
print(train_csv.info())

# 데이터셋을 결측치가 없는 데이터셋으로 초기화 
# 결측치 열 제거 
# 1. 결측치 처리 1. 제거 
train_csv = train_csv.dropna(axis=0)

# 결측치 가 있는 열 확인
train_csv.isnull().any()

# print(train_csv.shape)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1459 entries, 3 to 2179
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   hour                    1459 non-null   int64  
 1   hour_bef_temperature    1457 non-null   float64
 2   hour_bef_precipitation  1457 non-null   float64
 3   hour_bef_windspeed      1450 non-null   float64
 4   hour_bef_humidity       1457 non-null   float64
 5   hour_bef_visibility     1457 non-null   float64
 6   hour_bef_ozone          1383 non-null   float64
 7   hour_bef_pm10           1369 non-null   float64
 8   hour_bef_pm2.5          1342 non-null   float64
 9   count                   1459 non-null   float64
dtypes: float64(9), int64(1)
memory usage: 125.4 KB
None


hour                      False
hour_bef_temperature      False
hour_bef_precipitation    False
hour_bef_windspeed        False
hour_bef_humidity         False
hour_bef_visibility       False
hour_bef_ozone            False
hour_bef_pm10             False
hour_bef_pm2.5            False
count                     False
dtype: bool

In [18]:
# 데이터셋의 다양한 값들을 볼수는 describe
print(train_csv.describe())

              hour  hour_bef_temperature  hour_bef_precipitation  \
count  1328.000000           1328.000000             1328.000000   
mean     11.905120             16.866039                0.030120   
std       6.746907              5.196544                0.170983   
min       0.000000              3.100000                0.000000   
25%       6.000000             13.000000                0.000000   
50%      12.000000             16.800000                0.000000   
75%      18.000000             20.200000                0.000000   
max      23.000000             30.000000                1.000000   

       hour_bef_windspeed  hour_bef_humidity  hour_bef_visibility  \
count         1328.000000        1328.000000          1328.000000   
mean             2.489834          51.740964          1413.655120   
std              1.382797          20.167066           577.258705   
min              0.000000           7.000000            78.000000   
25%              1.400000          36.0000

In [19]:
# 데이터 정리하기 x 
x = train_csv.drop(["count"],axis=1)

#  x는 이제 count 칼럼이 사라졌습니다!
print(x.head())
print(x.shape)

    hour  hour_bef_temperature  hour_bef_precipitation  hour_bef_windspeed  \
id                                                                           
3     20                  16.3                     1.0                 1.5   
6     13                  20.1                     0.0                 1.4   
7      6                  13.9                     0.0                 0.7   
8     23                   8.1                     0.0                 2.7   
9     18                  29.5                     0.0                 4.8   

    hour_bef_humidity  hour_bef_visibility  hour_bef_ozone  hour_bef_pm10  \
id                                                                          
3                89.0                576.0           0.027           76.0   
6                48.0                916.0           0.042           73.0   
7                79.0               1382.0           0.033           32.0   
8                54.0                946.0           0.040          

In [20]:
# 데이터 정리하기 y
# count 칼럼만 가지고옵니다 이떄 train_csv 은 변형하지 않는게 포인트
y = train_csv["count"]
print(y.head())

print(y.shape)

id
3     49.0
6    159.0
7     26.0
8     57.0
9    431.0
Name: count, dtype: float64
(1328,)


In [21]:
# 데이터를 분할하기
x_train, x_test, y_train,y_test = train_test_split(x,y,
                                                   train_size=0.75,
                                                   shuffle = True,
                                                   random_state=21
                                                   )

print(len(x_train))
print(len(x_test))

996
332


In [22]:
# 2. 모델구성

model = Sequential([
    Dense(10,input_dim = 9),
    Dense(20, activation="relu"),
    Dense(40, activation="relu"),
    Dense(80, activation="relu"),
    Dense(80, activation="relu"),
    Dense(80, activation="relu"),
    Dense(160,activation="relu"),
    Dense(80, activation="relu"),
    Dense(80, activation="relu"),
    Dense(80, activation="relu"),
    Dense(40, activation="relu"),
    Dense(20, activation="relu"),
    Dense(10, activation="relu"),


    Dense(1)
])
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_10 (Dense)            (None, 10)                100       
                                                                 
 dense_11 (Dense)            (None, 20)                220       
                                                                 
 dense_12 (Dense)            (None, 40)                840       
                                                                 
 dense_13 (Dense)            (None, 80)                3280      
                                                                 
 dense_14 (Dense)            (None, 160)               12960     
                                                                 
 dense_15 (Dense)            (None, 80)                12880     
                                                                 
 dense_16 (Dense)            (None, 40)               

In [23]:
# 3.컴파일 + 훈련
# gpu 걸린시간 :  12.08470892906189
# cpu 걸린시간 :  16.177061557769775
import time

model.compile(loss="mse",optimizer="adam")
start = time.time()
model.fit(x_train,y_train,epochs=100,batch_size=3)
end = time.time()

print("걸린시간 : ",end-start)

Epoch 1/100
332/332 [==============================] - 1s 3ms/step - loss: 7300.7710
Epoch 2/100
332/332 [==============================] - 1s 3ms/step - loss: 6998.5913
Epoch 3/100
332/332 [==============================] - 1s 3ms/step - loss: 6168.4038
Epoch 4/100
332/332 [==============================] - 1s 3ms/step - loss: 5485.8638
Epoch 5/100
332/332 [==============================] - 1s 3ms/step - loss: 4577.5757
Epoch 6/100
332/332 [==============================] - 1s 3ms/step - loss: 3784.9380
Epoch 7/100
332/332 [==============================] - 1s 3ms/step - loss: 3603.5146
Epoch 8/100
332/332 [==============================] - 1s 3ms/step - loss: 3268.4502
Epoch 9/100
332/332 [==============================] - 1s 3ms/step - loss: 3384.2976
Epoch 10/100
332/332 [==============================] - 1s 3ms/step - loss: 3322.2798
Epoch 11/100
332/332 [==============================] - 1s 3ms/step - loss: 3521.8633
Epoch 12/100
332/332 [==============================] - 1s 3ms/

In [24]:
# 4. 성능평가

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

def RMSE(y_test,y_predict):
    return np.sqrt(mean_squared_error(y_test,y_predict))

loss = model.evaluate(x_test,y_test)
print("loss : ",loss)

y_predict =model.predict(x_test)
# print("=================")
# print(y_test)
# print(y_predict)
# print("=================")

print("RMSE : ",RMSE(y_test,y_predict))
print("R2 : ",r2_score(y_test,y_predict))

11/11 [==============================] - 0s 2ms/step - loss: 2916.2361
loss :  2916.236083984375
RMSE :  54.002186747253674
R2 :  0.5490574522503484


In [25]:
# 15. 저장
# 제출용 but NaN 이 포함되어있다 ...
import pandas as pd
# 15-1.제출용를 predcit 로 뽑아냄
y_submit = model.predict(test_csv)
# 15-2.제촐용을 데이터프레임형으로 변환
y_submit = pd.DataFrame(y_submit)
# 15-2 제출용의 카운트 칼럼에 y_submit 삽입
submission["count"] = y_submit
# 15-3 경로와 파일명 지정
submission.to_csv(path + "submission_0101502151.csv")